In [ ]:
import pickle
import numpy as np

user_path = '/mounts/work/weissweiler/deepcase/final_data/spacy_3.2_trf/'
silver = pickle.load(open("silver_standard.p", "rb"))

In [ ]:
def test_hyperparameters(min_freq, min_oddsratio, min_pvalue):
    ngrams_decided = {}
    filtered_ngrams_freq_oddsratio = pickle.load(open(user_path + 'ngrams_with_fisher.p', 'rb'))
    for prf in filtered_ngrams_freq_oddsratio.keys():

        ngrams_decided[prf] = set()
        filtered_prf = filtered_ngrams_freq_oddsratio[prf]
        for ngram, data in filtered_prf.items():
            (freq_inside, oddsratio, pvalue) = data

            if freq_inside > min_freq and oddsratio > min_oddsratio and pvalue < min_pvalue:
                if ngram[-1] == "$":
                    ngrams_decided[prf].add(ngram)

    precisions = []
    recalls = []
    fmeasures = []
    for lang, ngram_set in ngrams_decided.items():
        lang = lang[:3] #numbers come in from bible version
        if lang in silver: #3 languages that were taken out of the silver stndard late
            lang_silver = silver[lang]
            tp = ngram_set & lang_silver

            if len (ngram_set) > 0:
                precision = len(tp) / len(ngram_set)
                recall = len(tp) / len(lang_silver)
            else:
                precision = 0
                recall = 0

            if precision > 0 or recall > 0:
                fmeasure = 2*((precision * recall)/(precision + recall))
            else:
                fmeasure = 0

            precisions.append(precision)
            recalls.append(recall)
            fmeasures.append(fmeasure)
            
            print(lang, "PR", round(precision, 2), "RE", round(recall, 2), "F1", round(fmeasure,2))

    average_pr = sum(precisions)/len(precisions)
    average_re = sum(recalls)/len(recalls)
    average_f1 = sum(fmeasures)/len(fmeasures)

    print("average PR", round(average_pr, 2))
    print("average RE", round(average_re, 2))
    print("average F1", round(average_f1, 2))

    return average_f1, ngrams_decided

In [ ]:
for min_freq in range(90,100):
    for min_oddsratio in np.arange(0.3, 0.4, 0.01):
        for min_pvalue in np.arange(0.01, 0.1, 0.01):
            f1, _ = test_hyperparameters(min_freq, min_oddsratio, min_pvalue)
            print(f1, "|", min_freq, min_oddsratio, min_pvalue)

In [ ]:
min_freq = 96
min_oddsratio = 0.34
min_pvalue = 0.08
f1, ngrams = test_hyperparameters(min_freq, min_oddsratio, min_pvalue)
print(f1, "|", min_freq, min_oddsratio, min_pvalue)
pickle.dump(ngrams, open(user_path + 'found_case_markers.p', 'wb'))
